In [15]:
import torch
from dataloader import get_loader
from SGC_LSTM import SGC_LSTM
import argparse

In [16]:
parser = argparse.ArgumentParser()

# Model Configuration
parser.add_argument('--embedding_size',
                    type=int,
                    default=6,
                    help="嵌入的默认长度，以标准角色配置中的数目相同")

parser.add_argument('--layer_num',
                    type=int,
                    default=2,
                    help="sgcn层数")

parser.add_argument('--cell_num',
                    type=int,
                    default=25,
                    help="lstm的cell数量，最多为5轮任务，每轮任务5次投票，共计25次")

parser.add_argument('--lstm_input_size',
                    type=int,
                    default=66,
                    help="lstm的输入大小，默认为66（SGCN正嵌入32+SGCN负嵌入32+任务额外信息2）")

parser.add_argument('--lstm_hidden_size',
                    type=int,
                    default=32,
                    help="lstm的隐藏层大小，默认为32")

# Training Configuration
parser.add_argument('--batch_size',
                    type=int,
                    default=32)

parser.add_argument('--epoch_num',
                    type=int,
                    default=200)

# Directories
parser.add_argument('--record_dir',
                    type=str,
                    default='./data/gameRecordsDataAnon.json')

parser.add_argument('--data_dir',
                    type=str,
                    default='./data/simplifiedGameRecord/')
# 此处为jupyter独有的处理方式，原代码不用修改
config = parser.parse_known_args()[0]

In [17]:
def generate_graph(records):
    """
    Generate player graph for every game.
    :param records: Game record.
    :return edges: Graph list.
    """
    G = []  # 存储一个batch中所有游戏的图
    for record in records:
        graphs = []  # 存储一次游戏中所有图的列表
        game = record['gameProcess']  # 当前游戏
        playerNum = record["numberOfPlayers"]  # 玩家人数
        missionCount = 0
        for mission in game:  # 当前任务
            missionGraphs = []  # 存储一次任务中所有图的列表
            for vote in mission:  # 当前投票
                role = vote[0]  # 玩家角色
                voteResult = vote[1]  # 投票结果
                # 存储当前投票结果对应的图的字典
                voteGraph = {"numberOfPlayers": record["numberOfPlayers"], "positiveEdges": [], "negativeEdges": [],
                             "Members": [], "nonMembers": [], "missionResult": record["missionHistory"][missionCount],
                             "rolesNum": record["rolesNum"]}
                for k in range(playerNum):
                    player = role[k]
                    if player == "Member":  # 之后改一下数据集，让变量命名一致
                        voteGraph["Members"].append(k)
                    elif player == "nonMember":
                        voteGraph["nonMembers"].append(k)
                    else:
                        voteGraph["Leader"] = k
                        if player == "MemberLeader":
                            voteGraph["Members"].append(k)
                        else:
                            voteGraph["nonMembers"].append(k)

                for k in range(playerNum):
                    for m in range(playerNum):
                        if k == m:
                            continue
                        elif voteResult[k] == voteResult[m]:
                            voteGraph["positiveEdges"].append([k, m])
                        else:
                            voteGraph["negativeEdges"].append([k, m])
                missionGraphs.append(voteGraph)
            missionCount += 1
            graphs.append(missionGraphs)
        G.append(graphs)
    return G

In [21]:
import numpy as np
from signed_sage_convolution import SignedSAGEConvolutionBase, SignedSAGEConvolutionDeep, ListModule
from utils import initialize_embedding, padding, judge

class SGC_LSTM(torch.nn.Module):
    """
    SGC_LSTM NetWork Class.
    """

    def __init__(self, device, config):
        """
        Initialize SGC_LSTM.
        :param device: Device for calculations.
        :param config: Arguments object.
        """
        super(SGC_LSTM, self).__init__()
        # 参数对象
        self.device = device
        self.config = config
        self.cell_num = self.config.cell_num
        self.lstm = []

        self.setup_layers()

    def setup_sgcn(self):
        """
        搭建sgcn
        """
        # 输入是28*1（7*4，3种一维邻居的聚合结果7，一共21，自己7，拼接后28）的向量
        self.positive_base_aggregator = SignedSAGEConvolutionBase(self.config.embedding_size * 4, 32).to(self.device)
        self.negative_base_aggregator = SignedSAGEConvolutionBase(self.config.embedding_size * 4, 32).to(self.device)

        self.positive_aggregators = []
        self.negative_aggregators = []
        for i in range(2):
            # 输入是32*7（6种+自己）输出暂定32*1
            self.positive_aggregators.append(SignedSAGEConvolutionDeep(32 * 7, 32).to(self.device))

            self.negative_aggregators.append(SignedSAGEConvolutionDeep(32 * 7, 32).to(self.device))

        self.positive_aggregators = ListModule(*self.positive_aggregators)
        self.negative_aggregators = ListModule(*self.negative_aggregators)

    def setup_layers(self):
        # setup_sgcn只能生成满足一轮任务的sgcn，应该调用和游戏轮数相同次，每个游戏用的不一样
        # sgcn中未包含激活函数
        self.setup_sgcn()
        for _ in range(self.cell_num):
            self.lstm.append(torch.nn.LSTMCell(self.config.lstm_input_size, self.config.lstm_hidden_size))
        self.lstm_list = ListModule(*self.lstm)
        self.W = torch.nn.Linear(self.config.lstm_hidden_size, self.config.embedding_size)
        self.m = torch.nn.Softmax(dim=0)

    def forward(self, graphs):
        """

        :param graphs:
        :return:
        """
        # 先串行进行SGCN计算
        # game是单独一局游戏
        out = []
        for game in graphs:
            player_num = game[0][0]["numberOfPlayers"]
            embedding_size = self.config.embedding_size
            game_embedding, game_embedding_list = [], []

            for _ in range(player_num):
                game_embedding_list.append([])

            for mission in game:
                add_info = []  # 存储每个玩家的任务额外信息（任务成功，是否参与任务）
                mission_embedding_list = []  # 临时存储任务中每一轮投票的嵌入,二维列表，第一维为角色

                for _ in range(player_num):
                    add_info.append(torch.ones(2))
                    mission_embedding_list.append([])

                for vote in mission:
                    # 生成初始嵌入
                    h_0 = []  # 初始嵌入
                    for _ in range(player_num):
                        mission_embedding_list.append([])
                        h_0.append(initialize_embedding(embedding_size))
                    # 目前sgcn接收一轮投票的图，返回所有玩家当前的嵌入
                    # 因此这里还需要加一个玩家的维度
                    # h_pos[i]表示第i次聚合后所有点的嵌入的列表
                    h_pos, h_neg = [], []
                    # 进行第一层SGCN
                    h_pos.append(self.positive_base_aggregator(vote, "positive", h_0))
                    h_neg.append(self.negative_base_aggregator(vote, "negative", h_0))

                    # 第二层SGCN
                    for i in range(1, self.config.layer_num):
                        self.h_pos.append(
                            self.positive_aggregators[i - 1](vote, "positive", h_pos[i - 1], h_neg[i - 1]))
                        self.h_pos.append(
                            self.negative_aggregators[i - 1](vote, "negative", h_pos[i - 1], h_neg[i - 1]))
                    # h_pos[i] player_num个1*32tensor

                    for player in range(player_num):
                        mission_embedding_list[player].append(
                            torch.cat((h_pos[-1][player], h_neg[-1][player]), 1))  # player_num个1*64tensor
                        # 修改add_info
                        if player not in vote["nonMember"]:
                            add_info[player][1] = 0  # 任意一轮投票中未参与组队，修改add_info
                        add_info[player][0] = vote["missionResult"]  # 添加任务结果，每个角色只用只添加一次

                # 对当前任务进行padding（填充一个全为-1的向量），获得一个5*66(输出64+额外添加的2)的向量
                for player in range(player_num):
                    #  添加补充信息
                    for i in range(len(mission_embedding_list[player])):
                        mission_embedding_list[player][i] = torch.cat(
                            (mission_embedding_list[player][i], add_info[player]), 1)
                    # padding
                    for i in range(5):
                        # 有对应的投票，直接复制
                        if i < len(mission_embedding_list[player]):
                            game_embedding_list[player].append(mission_embedding_list[player][i])
                        # 否则添加全为-1的tensor
                        else:
                            game_embedding_list[player].append(torch.full((1, self.config.lstm_input_size), -1))

            h_final = []  # 最终嵌入的列表
            for player in range(player_num):
                embedding = padding(game_embedding_list[player], 25,
                                    self.config.lstm_input_size)  # 一个玩家在该轮任务中的嵌入，形状为25*66
                hx = torch.randn(1, self.config.lstm_hidden_size)  # 初始化方式待讨论
                cx = torch.randn(1, self.config.lstm_hidden_size)
                for i in range(self.cell_num):
                    if not judge(embedding[i]):  # 判断是不是全是-1
                        cell = self.lstm_list[i]
                        hx, cx = cell(embedding[i], (hx, cx))  # 待讨论
                h_final.append(self.m(self.W(hx)))  # Softmax归一化

            h_out = []
            # 本局游戏的预测结果
            for h in h_final:
                role = torch.zeros(embedding_size)
                x = h.numpy()
                pos = np.argmax(x)
                role[pos] = 1
                h_out.append(role)

            out.append(h_out)

In [19]:
data_loader = get_loader(config, 'train')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SGC_LSTM(device, config)
model.to(device)
for i, data in enumerate(data_loader):
    records = data[0]
    labels = data[1]
    break

In [25]:
graphs = generate_graph(records)  # 一个batch中所有游戏的图
print(graphs)
out = model(graphs)
print(out)

[[[{'numberOfPlayers': 6, 'positiveEdges': [[0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1], [3, 4], [3, 5], [4, 3], [4, 5], [5, 3], [5, 4]], 'negativeEdges': [[0, 3], [0, 4], [0, 5], [1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [2, 5], [3, 0], [3, 1], [3, 2], [4, 0], [4, 1], [4, 2], [5, 0], [5, 1], [5, 2]], 'Members': [1, 2], 'nonMembers': [0, 3, 4, 5], 'missionResult': 1, 'rolesNum': [2.0, 0.0, 1.0, 1.0, 1.0, 1.0], 'Leader': 1}, {'numberOfPlayers': 6, 'positiveEdges': [[0, 2], [0, 5], [1, 3], [1, 4], [2, 0], [2, 5], [3, 1], [3, 4], [4, 1], [4, 3], [5, 0], [5, 2]], 'negativeEdges': [[0, 1], [0, 3], [0, 4], [1, 0], [1, 2], [1, 5], [2, 1], [2, 3], [2, 4], [3, 0], [3, 2], [3, 5], [4, 0], [4, 2], [4, 5], [5, 1], [5, 3], [5, 4]], 'Members': [2, 5], 'nonMembers': [0, 1, 3, 4], 'missionResult': 1, 'rolesNum': [2.0, 0.0, 1.0, 1.0, 1.0, 1.0], 'Leader': 0}, {'numberOfPlayers': 6, 'positiveEdges': [[0, 4], [0, 5], [1, 2], [1, 3], [2, 1], [2, 3], [3, 1], [3, 2], [4, 0], [4, 5], [5, 0], [5, 4]], 'negati

RuntimeError: The size of tensor a (7) must match the size of tensor b (6) at non-singleton dimension 1

In [ ]:
data_loader = get_loader(config, 'train')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SGC_LSTM(device, config)
print(model)
model.to(device)

# enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标
for i, data in enumerate(data_loader):
    # 第一维为完整游戏记录，第二维为玩家真实身份，注意是一个batch的数据
    records = data[0]
    labels = data[1]
    graphs = generate_graph(records)  # 一个batch中所有游戏的图
    out = model(graphs)
    print(out)